In [38]:
import pandas as pd
import json
import sklearn.svm as svm
import numpy as np
from sklearn.neural_network import MLPClassifier

In [27]:
## Change the name to get your path
user = 'Yash'
file1 = 'AG01-01.csv'
file2 = 'GTH01-01.csv'

In [28]:
def load_Data(user, filename):
   data = json.load(open('dir.json'))
   dir = data[user] + filename
#    data = pd.read_csv(dir, header=2)
   return dir

In [112]:
def useable_Data(data, gt):
    data['start.time'] = pd.to_datetime(data['start.time'])
    observedData = data[(data['start.time'] >= '2017-10-03 16:44:00') & (data['start.time'] <= '2017-10-03 18:44:00')]
    observedData['index'] = range(len(observedData))
    gt['index'] = gt['time']
    observedData = pd.merge(observedData,gt[['index','coding']])
    observedData.sort_index(inplace=True)
    copy = observedData[['mean.vm','sd.vm','mean.ang','sd.ang','p625','dfreq','ratio.df']].copy()
    copy.loc[-1] = copy.loc[0]  # adding a row
    copy.index = copy.index + 1  # shifting index
    copy.sort_index(inplace=True)
    copy.columns = 'last.' + copy.columns
    observedData = pd.concat([observedData, copy], axis = 1)
    observedData = observedData.drop(observedData.index[len(observedData)-1])
    return observedData
#     return copy

In [ ]:
data = pd.read_csv(load_Data(user, file1), header=0)
gt = pd.read_csv(load_Data(user, file2), header=0)
observedData = pd.DataFrame(useable_Data(data, gt))
observedData.head()

In [ ]:
observedData = observedData.sample(frac = 1)

train = observedData[:5000]
test = observedData[5000:]

## this might be an issue since there is ordinality and that makes things weird.  The later parts of this trial
## were more likely to be sedentary.
X = observedData[['mean.vm','sd.vm','mean.ang','sd.ang','p625','dfreq','ratio.df', 'last.mean.vm', 'last.sd.vm', 'last.mean.ang', 'last.sd.ang', 'last.p625', 'last.dfreq', 'last.ratio.df']]
y = observedData['coding']

trainX = train[['mean.vm','sd.vm','mean.ang','sd.ang','p625','dfreq','ratio.df', 'last.mean.vm', 'last.sd.vm', 'last.mean.ang', 'last.sd.ang', 'last.p625', 'last.dfreq', 'last.ratio.df']]
trainY = train['coding']
testX = test[['mean.vm','sd.vm','mean.ang','sd.ang','p625','dfreq','ratio.df', 'last.mean.vm', 'last.sd.vm', 'last.mean.ang', 'last.sd.ang', 'last.p625', 'last.dfreq', 'last.ratio.df']]
testY = test['coding']

## SVM

In [ ]:
clf = svm.SVC(cache_size=7000)
clf.fit(trainX, trainY)

In [39]:
clf.score(X,y)

0.7923151081118304

## Neural Network

In [ ]:
n_net = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(2000, 1000), random_state=1)
n_net.fit(X,y)